In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
import os
os.chdir('/content/gdrive/My Drive/')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# os.chdir('/content/gdrive/My Drive/testpy')

# %run __init__.py
# # Test().test()
# # print(dir(Test))
# # Test().test1()

In [0]:
import torch
import cv2
import numpy as np
import os.path as osp
import os
import sys
import torch.utils.data as data

In [0]:
# config.py
import os.path

# gets home dir cross platform
HOME = os.path.expanduser("~")

# for making bounding boxes pretty
COLORS = ((255, 0, 0, 128), (0, 255, 0, 128), (0, 0, 255, 128),
          (0, 255, 255, 128), (255, 0, 255, 128), (255, 255, 0, 128))

MEANS = (104, 117, 123)

# SSD300 CONFIGS
voc = {
    'num_classes': 21,
    'lr_steps': (80000, 100000, 120000),
    'max_iter': 120000,
    'feature_maps': [38, 19, 10, 5, 3, 1],
    'min_dim': 300,
    'steps': [8, 16, 32, 64, 100, 300],
    'min_sizes': [30, 60, 111, 162, 213, 264],
    'max_sizes': [60, 111, 162, 213, 264, 315],
    'aspect_ratios': [[2], [2, 3], [2, 3], [2, 3], [2], [2]],
    'variance': [0.1, 0.2],
    'clip': True,
    'name': 'VOC',
}

coco = {
    'num_classes': 201,
    'lr_steps': (280000, 360000, 400000),
    'max_iter': 400000,
    'feature_maps': [38, 19, 10, 5, 3, 1],
    'min_dim': 300,
    'steps': [8, 16, 32, 64, 100, 300],
    'min_sizes': [21, 45, 99, 153, 207, 261],
    'max_sizes': [45, 99, 153, 207, 261, 315],
    'aspect_ratios': [[2], [2, 3], [2, 3], [2, 3], [2], [2]],
    'variance': [0.1, 0.2],
    'clip': True,
    'name': 'COCO',
}

_sixray = {
    'num_classes': 3,
    'lr_steps': (8000, 10000, 12000),
    'max_iter': 12000,
    'feature_maps': [38, 19, 10, 5, 3, 1],
    'min_dim': 300,
    'steps': [8, 16, 32, 64, 100, 300],
    'min_sizes': [30, 60, 111, 162, 213, 264],
    'max_sizes': [60, 111, 162, 213, 264, 315],
    'aspect_ratios': [[2], [2, 3], [2, 3], [2, 3], [2], [2]],
    'variance': [0.1, 0.2],
    'clip': True,
    'name': 'VOC',
}


In [0]:
SIXray_CLASSES = (
    '带电芯充电宝', '不带电芯充电宝'
)
# note: if you used our download scripts, this should be right
SIXray_ROOT = "/content/gdrive/My Drive/all/"


class SIXrayAnnotationTransform(object):
    """Transforms a VOC annotation into a Tensor of bbox coords and label index
    Initilized with a dictionary lookup of classnames to indexes

    Arguments:
        class_to_ind (dict, optional): dictionary lookup of classnames -> indexes
            (default: alphabetic indexing of VOC's 20 classes)
        keep_difficult (bool, optional): keep difficult instances or not
            (default: False)
        height (int): height
        width (int): width
    """

    def __init__(self, class_to_ind=None, keep_difficult=False):
        self.class_to_ind = class_to_ind or dict(
            # zip(SIXray_CLASSES, range(len(SIXray_CLASSES))))
            zip(SIXray_CLASSES, range(len(SIXray_CLASSES))))
        self.keep_difficult = keep_difficult
        # 添加的记录所有小类总数
        self.type_dict = {}
        # 记录大类数量
        self.type_sum_dict = {}

    def __call__(self, target, width, height, idx):
        """
        Arguments:
            target (annotation) : the target annotation to be made usable
                will be an ET.Element
            it has been changed to the path of annotation-2019-07-10
        Returns:
            a list containing lists of bounding boxes  [bbox coords, class name]
        """
        # print (idx)

        ###读取所有类
        # class_path = "C:/Users/石玉峰/PycharmProjects/untitled1/package2/class.xml"
        '''
        class_path = SIXray_ROOT + "class.xml"
        tree = ET.parse(class_path)
        class_root = tree.getroot()
        for child in class_root:
            one_dict = {}
            type_name = None
            find_name = 0
            for grandson in child:
                if find_name == 0:
                    find_name = 1
                    type_name = grandson.text
                else:
                    one_dict[grandson.text] = 0
            self.type_dict[type_name] = one_dict
        #print(self.type_dict)
        '''
        # 遍历Annotation
        # root_annotation = '/media/dsg3/datasets/Xray20190704/Annotation/'
        res = []
        with open(target, "r", encoding='utf-8') as f1:
            dataread = f1.readlines()
        for annotation in dataread:
            bndbox = []
            temp = annotation.split()
            name = temp[1]
            # 只读两类
            if name != '带电芯充电宝' and name != '不带电芯充电宝':
                continue
            xmin = int(temp[2]) / width
            # 只读取V视角的
            if xmin > 1:
                continue
            if xmin < 0:
                xmin = 0
            ymin = int(temp[3]) / height
            if ymin < 0:
                ymin = 0
            xmax = int(temp[4]) / width
            if xmax > 1:  # 是这么个意思吧？
                xmax = 1
            ymax = int(temp[5]) / height
            if ymax > 1:
                ymax = 1
            bndbox.append(xmin)
            bndbox.append(ymin)
            bndbox.append(xmax)
            bndbox.append(ymax)
            label_idx = self.class_to_ind[name]
            # label_idx = name
            bndbox.append(label_idx)
            res += [bndbox]  # [xmin, ymin, xmax, ymax, label_ind]
        if len(res) == 0:
            return [[0, 0, 0, 0, 3]]
        return res


class SIXrayDetection(data.Dataset):
    """VOC Detection Dataset Object

    input is image, target is annotation

    Arguments:
        root (string): filepath to VOCdevkit folder.
        image_set (string): imageset to use (eg. 'train', 'val', 'test')
        transform (callable, optional): transformation to perform on the
            input image
        target_transform (callable, optional): transformation to perform on the
            target `annotation`
            (eg: take in caption string, return tensor of word indices)
        dataset_name (string, optional): which dataset to load
            (default: 'VOC2007')
    """

    def __init__(self, root,
                 image_sets,
                 transform=None, target_transform=SIXrayAnnotationTransform(),
                 # dataset_name='SIXray'):
                 dataset_name='SIXray', phase='train'):
        # self.root = root
        self.name = 'sixray'
        self.root = SIXray_ROOT
        self.image_set = image_sets
        self.transform = transform
        self.target_transform = target_transform
        # self.name = dataset_name
        #self.name = 'Xray0723_bat_core_coreless'
        self._annopath = osp.join('%s' % self.root, 'Annotation', '%s.txt')
        self._imgpath = osp.join('%s' % self.root, 'Image', '%s.jpg')
        # self._annopath = osp.join('%s' % self.root, 'Annotation', '%s.xml')
        # self._annopath = osp.join('%s' % self.root, 'Anno_core_coreless_battery_sub_2000_500', '%s.txt')
        # self._imgpath = osp.join('%s' % self.root, 'Image', '%s.jpg')
        # self._imgpath = osp.join('%s' % self.root, 'cut_Image_core_coreless_battery_sub_2000_500', '%s.TIFF')
        ###这尼玛还有小写的tiff？
        # self._imgpath1 = osp.join('%s' % self.root, 'cut_Image_core_coreless_battery_sub_2000_500', '%s.tiff')
        # self._imgpath_jpg = osp.join('%s' % self.root, 'cut_Image_core_coreless_battery_sub_2000_500', '%s.jpg')
        self.ids = list()

        listdir = os.listdir(osp.join('%s' % self.root, 'Annotation'))
        # testdir = listdir[0:6000:5]
        testdir = listdir[0:6000:10]
        print(testdir)

        # with open(self.image_set, 'r') as f:
        #     lines = f.readlines()
        #     for line in lines:
        #         self.ids.append(line.strip('\n'))
        if phase == 'train':
          traindir = set(listdir).difference(set(testdir))
          # traindir = list(traindir)[0:1200]
          for name in traindir:
              self.ids.append(osp.splitext(name)[0])
        else:
          for name in testdir:
              self.ids.append(osp.splitext(name)[0])

    def __getitem__(self, index):
        im, gt, h, w, og_im = self.pull_item(index)

        return im, gt

    def __len__(self):
        return len(self.ids)

    def pull_item(self, index):
        img_id = self.ids[index]

        # target = ET.parse(self._annopath % img_id).getroot()

        target = self._annopath % img_id  # 注释目录
        # print(target)
        # print(self._imgpath % img_id)
        img = cv2.imread(self._imgpath % img_id)
        # if img is None:
        #     img = cv2.imread(self._imgpath1 % img_id)
        # if img is None:
        #     img = cv2.imread(self._imgpath_jpg % img_id)

        if img is None:
            print('\nwrong\n')
            # print(self._imgpath_jpg % img_id)

        # print()
        height, width, channels = img.shape
        # print("height: " + str(height) + " ; width : " + str(width) + " ; channels " + str(channels) )
        og_img = img

        # print (img_id)
        if self.target_transform is not None:
            target = self.target_transform(target, width, height, img_id)

        if self.transform is not None:
            target = np.array(target)
            # print(target)
            img, boxes, labels = self.transform(img, target[:, :4], target[:, 4])
            # to rgb
            img = img[:, :, (2, 1, 0)]
            # img = img.transpose(a2, 0, a1)
            target = np.hstack((boxes, np.expand_dims(labels, axis=1)))
        return torch.from_numpy(img).permute(2, 0, 1), target, height, width, og_img

In [0]:

def detection_collate(batch):
    """Custom collate fn for dealing with batches of images that have a different
    number of associated object annotations (bounding boxes).

    Arguments:
        batch: (tuple) A tuple of tensor images and lists of annotations

    Return:
        A tuple containing:
            1) (tensor) batch of images stacked on their 0 dim
            2) (list of tensors) annotations for a given image are stacked on
                                 0 dim
    """
    targets = []
    imgs = []
    for sample in batch:
        imgs.append(sample[0])
        targets.append(torch.FloatTensor(sample[1]))
    return torch.stack(imgs, 0), targets


def base_transform(image, size, mean):
    x = cv2.resize(image, (size, size)).astype(np.float32)
    x -= mean
    x = x.astype(np.float32)
    return x


class BaseTransform:
    def __init__(self, size, mean):
        self.size = size
        self.mean = np.array(mean, dtype=np.float32)

    def __call__(self, image, boxes=None, labels=None):
        return base_transform(image, self.size, self.mean), boxes, labels


In [0]:

def point_form(boxes):
    """ Convert prior_boxes to (xmin, ymin, xmax, ymax)
    representation for comparison to point form ground truth data.
    Args:
        boxes: (tensor) center-size default boxes from priorbox layers.
    Return:
        boxes: (tensor) Converted xmin, ymin, xmax, ymax form of boxes.
    """
    return torch.cat((boxes[:, :2] - boxes[:, 2:]/2,     # xmin, ymin
                     boxes[:, :2] + boxes[:, 2:]/2), 1)  # xmax, ymax


def center_size(boxes):
    """ Convert prior_boxes to (cx, cy, w, h)
    representation for comparison to center-size form ground truth data.
    Args:
        boxes: (tensor) point_form boxes
    Return:
        boxes: (tensor) Converted xmin, ymin, xmax, ymax form of boxes.
    """
    return torch.cat((boxes[:, 2:] + boxes[:, :2])/2,  # cx, cy
                     boxes[:, 2:] - boxes[:, :2], 1)  # w, h


def intersect(box_a, box_b):
    """ We resize both tensors to [A,B,2] without new malloc:
    [A,2] -> [A,1,2] -> [A,B,2]
    [B,2] -> [1,B,2] -> [A,B,2]
    Then we compute the area of intersect between box_a and box_b.
    Args:
      box_a: (tensor) bounding boxes, Shape: [A,4].
      box_b: (tensor) bounding boxes, Shape: [B,4].
    Return:
      (tensor) intersection area, Shape: [A,B].
    """
    A = box_a.size(0)
    B = box_b.size(0)
    max_xy = torch.min(box_a[:, 2:].unsqueeze(1).expand(A, B, 2),
                       box_b[:, 2:].unsqueeze(0).expand(A, B, 2))
    min_xy = torch.max(box_a[:, :2].unsqueeze(1).expand(A, B, 2),
                       box_b[:, :2].unsqueeze(0).expand(A, B, 2))
    inter = torch.clamp((max_xy - min_xy), min=0)
    return inter[:, :, 0] * inter[:, :, 1]


def jaccard(box_a, box_b):
    """Compute the jaccard overlap of two sets of boxes.  The jaccard overlap
    is simply the intersection over union of two boxes.  Here we operate on
    ground truth boxes and default boxes.
    E.g.:
        A ∩ B / A ∪ B = A ∩ B / (area(A) + area(B) - A ∩ B)
    Args:
        box_a: (tensor) Ground truth bounding boxes, Shape: [num_objects,4]
        box_b: (tensor) Prior boxes from priorbox layers, Shape: [num_priors,4]
    Return:
        jaccard overlap: (tensor) Shape: [box_a.size(0), box_b.size(0)]
    """
    inter = intersect(box_a, box_b)
    area_a = ((box_a[:, 2]-box_a[:, 0]) *
              (box_a[:, 3]-box_a[:, 1])).unsqueeze(1).expand_as(inter)  # [A,B]
    area_b = ((box_b[:, 2]-box_b[:, 0]) *
              (box_b[:, 3]-box_b[:, 1])).unsqueeze(0).expand_as(inter)  # [A,B]
    union = area_a + area_b - inter
    return inter / union  # [A,B]


def match(threshold, truths, priors, variances, labels, loc_t, conf_t, idx):
    """Match each prior box with the ground truth box of the highest jaccard
    overlap, encode the bounding boxes, then return the matched indices
    corresponding to both confidence and location preds.
    Args:
        threshold: (float) The overlap threshold used when mathing boxes.
        truths: (tensor) Ground truth boxes, Shape: [num_obj, num_priors].
        priors: (tensor) Prior boxes from priorbox layers, Shape: [n_priors,4].
        variances: (tensor) Variances corresponding to each prior coord,
            Shape: [num_priors, 4].
        labels: (tensor) All the class labels for the image, Shape: [num_obj].
        loc_t: (tensor) Tensor to be filled w/ endcoded location targets.
        conf_t: (tensor) Tensor to be filled w/ matched indices for conf preds.
        idx: (int) current batch index
    Return:
        The matched indices corresponding to 1)location and 2)confidence preds.
    """
    # jaccard index
    overlaps = jaccard(
        truths,
        point_form(priors)
    )
    # (Bipartite Matching)
    # [1,num_objects] best prior for each ground truth
    best_prior_overlap, best_prior_idx = overlaps.max(1, keepdim=True)
    # [1,num_priors] best ground truth for each prior
    best_truth_overlap, best_truth_idx = overlaps.max(0, keepdim=True)
    best_truth_idx.squeeze_(0)
    best_truth_overlap.squeeze_(0)
    best_prior_idx.squeeze_(1)
    best_prior_overlap.squeeze_(1)
    best_truth_overlap.index_fill_(0, best_prior_idx, 2)  # ensure best prior
    # TODO refactor: index  best_prior_idx with long tensor
    # ensure every gt matches with its prior of max overlap
    for j in range(best_prior_idx.size(0)):
        best_truth_idx[best_prior_idx[j]] = j
    matches = truths[best_truth_idx]          # Shape: [num_priors,4]
    conf = labels[best_truth_idx] + 1         # Shape: [num_priors]
    conf[best_truth_overlap < threshold] = 0  # label as background
    loc = encode(matches, priors, variances)
    loc_t[idx] = loc    # [num_priors,4] encoded offsets to learn
    conf_t[idx] = conf  # [num_priors] top class label for each prior


def encode(matched, priors, variances):
    """Encode the variances from the priorbox layers into the ground truth boxes
    we have matched (based on jaccard overlap) with the prior boxes.
    Args:
        matched: (tensor) Coords of ground truth for each prior in point-form
            Shape: [num_priors, 4].
        priors: (tensor) Prior boxes in center-offset form
            Shape: [num_priors,4].
        variances: (list[float]) Variances of priorboxes
    Return:
        encoded boxes (tensor), Shape: [num_priors, 4]
    """

    # dist b/t match center and prior's center
    g_cxcy = (matched[:, :2] + matched[:, 2:])/2 - priors[:, :2]
    # encode variance
    g_cxcy /= (variances[0] * priors[:, 2:])
    # match wh / prior wh
    g_wh = (matched[:, 2:] - matched[:, :2]) / priors[:, 2:]
    g_wh = torch.log(g_wh) / variances[1]
    # return target for smooth_l1_loss
    return torch.cat([g_cxcy, g_wh], 1)  # [num_priors,4]


# Adapted from https://github.com/Hakuyume/chainer-ssd
def decode(loc, priors, variances):
    """Decode locations from predictions using priors to undo
    the encoding we did for offset regression at train time.
    Args:
        loc (tensor): location predictions for loc layers,
            Shape: [num_priors,4]
        priors (tensor): Prior boxes in center-offset form.
            Shape: [num_priors,4].
        variances: (list[float]) Variances of priorboxes
    Return:
        decoded bounding box predictions
    """

    boxes = torch.cat((
        priors[:, :2] + loc[:, :2] * variances[0] * priors[:, 2:],
        priors[:, 2:] * torch.exp(loc[:, 2:] * variances[1])), 1)
    boxes[:, :2] -= boxes[:, 2:] / 2
    boxes[:, 2:] += boxes[:, :2]
    return boxes


def log_sum_exp(x):
    """Utility function for computing log_sum_exp while determining
    This will be used to determine unaveraged confidence loss across
    all examples in a batch.
    Args:
        x (Variable(tensor)): conf_preds from conf layers
    """
    x_max = x.data.max()
    return torch.log(torch.sum(torch.exp(x-x_max), 1, keepdim=True)) + x_max


# Original author: Francisco Massa:
# https://github.com/fmassa/object-detection.torch
# Ported to PyTorch by Max deGroot (02/01/2017)
def nms(boxes, scores, overlap=0.5, top_k=200):
    """Apply non-maximum suppression at test time to avoid detecting too many
    overlapping bounding boxes for a given object.
    Args:
        boxes: (tensor) The location preds for the img, Shape: [num_priors,4].
        scores: (tensor) The class predscores for the img, Shape:[num_priors].
        overlap: (float) The overlap thresh for suppressing unnecessary boxes.
        top_k: (int) The Maximum number of box preds to consider.
    Return:
        The indices of the kept boxes with respect to num_priors.
    """

    keep = scores.new(scores.size(0)).zero_().long()
    if boxes.numel() == 0:
        return keep
    x1 = boxes[:, 0]
    y1 = boxes[:, 1]
    x2 = boxes[:, 2]
    y2 = boxes[:, 3]
    area = torch.mul(x2 - x1, y2 - y1)
    v, idx = scores.sort(0)  # sort in ascending order
    # I = I[v >= 0.01]
    idx = idx[-top_k:]  # indices of the top-k largest vals
    xx1 = boxes.new()
    yy1 = boxes.new()
    xx2 = boxes.new()
    yy2 = boxes.new()
    w = boxes.new()
    h = boxes.new()

    # keep = torch.Tensor()
    count = 0
    while idx.numel() > 0:
        i = idx[-1]  # index of current largest val
        # keep.append(i)
        keep[count] = i
        count += 1
        if idx.size(0) == 1:
            break
        idx = idx[:-1]  # remove kept element from view
        # load bboxes of next highest vals
        torch.index_select(x1, 0, idx, out=xx1)
        torch.index_select(y1, 0, idx, out=yy1)
        torch.index_select(x2, 0, idx, out=xx2)
        torch.index_select(y2, 0, idx, out=yy2)
        # store element-wise max with next highest score
        xx1 = torch.clamp(xx1, min=x1[i])
        yy1 = torch.clamp(yy1, min=y1[i])
        xx2 = torch.clamp(xx2, max=x2[i])
        yy2 = torch.clamp(yy2, max=y2[i])
        w.resize_as_(xx2)
        h.resize_as_(yy2)
        w = xx2 - xx1
        h = yy2 - yy1
        # check sizes of xx1 and xx2.. after each iteration
        w = torch.clamp(w, min=0.0)
        h = torch.clamp(h, min=0.0)
        inter = w*h
        # IoU = i / (area(a) + area(b) - i)
        rem_areas = torch.index_select(area, 0, idx)  # load remaining areas)
        union = (rem_areas - inter) + area[i]
        IoU = inter/union  # store result in iou
        # keep only elements with an IoU <= overlap
        idx = idx[IoU.le(overlap)]
    return keep, count


In [0]:
from torch.autograd import Function

class Detect(Function):
    """At test time, Detect is the final layer of SSD.  Decode location preds,
    apply non-maximum suppression to location predictions based on conf
    scores and threshold to a top_k number of output predictions for both
    confidence score and locations.
    """
    def __init__(self, num_classes, bkg_label, top_k, conf_thresh, nms_thresh):
        self.num_classes = num_classes
        self.background_label = bkg_label
        self.top_k = top_k
        # Parameters used in nms.
        self.nms_thresh = nms_thresh
        if nms_thresh <= 0:
            raise ValueError('nms_threshold must be non negative.')
        self.conf_thresh = conf_thresh
        self.variance = cfg['variance']

    def forward(self, *args):
        """
        Args:
            loc_data: (tensor) Loc preds from loc layers
                Shape: [batch,num_priors*4]
            conf_data: (tensor) Shape: Conf preds from conf layers
                Shape: [batch*num_priors,num_classes]
            prior_data: (tensor) Prior boxes and variances from priorbox layers
                Shape: [1,num_priors,4]
        """
        loc_data = args[0]
        conf_data = args[1]
        prior_data = args[2]
        num = loc_data.size(0)  # batch size
        num_priors = prior_data.size(0)
        output = torch.zeros(num, self.num_classes, self.top_k, 5)
        conf_preds = conf_data.view(num, num_priors,
                                    self.num_classes).transpose(2, 1)

        # Decode predictions into bboxes.
        for i in range(num):
            decoded_boxes = decode(loc_data[i], prior_data, self.variance)
            # For each class, perform nms
            conf_scores = conf_preds[i].clone()

            for cl in range(1, self.num_classes):
                c_mask = conf_scores[cl].gt(self.conf_thresh)
                scores = conf_scores[cl][c_mask]
                if scores.size(0) == 0:
                    continue
                l_mask = c_mask.unsqueeze(1).expand_as(decoded_boxes)
                boxes = decoded_boxes[l_mask].view(-1, 4)
                # idx of highest scoring and non-overlapping boxes per class
                ids, count = nms(boxes, scores, self.nms_thresh, self.top_k)
                output[i, cl, :count] = \
                    torch.cat((scores[ids[:count]].unsqueeze(1),
                               boxes[ids[:count]]), 1)
        flt = output.contiguous().view(num, -1, 5)
        _, idx = flt[:, :, 0].sort(1, descending=True)
        _, rank = idx.sort(1)
        flt[(rank < self.top_k).unsqueeze(-1).expand_as(flt)].fill_(0)
        return output


In [0]:
from __future__ import division
from math import sqrt as sqrt
from itertools import product as product
import torch


class PriorBox(object):
    """Compute priorbox coordinates in center-offset form for each source
    feature map.
    """
    def __init__(self, cfg):
        super(PriorBox, self).__init__()
        self.image_size = cfg['min_dim']
        # number of priors for feature map location (either 4 or 6)
        self.num_priors = len(cfg['aspect_ratios'])
        self.variance = cfg['variance'] or [0.1]
        self.feature_maps = cfg['feature_maps']
        self.min_sizes = cfg['min_sizes']
        self.max_sizes = cfg['max_sizes']
        self.steps = cfg['steps']
        self.aspect_ratios = cfg['aspect_ratios']
        self.clip = cfg['clip']
        self.version = cfg['name']
        for v in self.variance:
            if v <= 0:
                raise ValueError('Variances must be greater than 0')

    def forward(self):
        mean = []
        for k, f in enumerate(self.feature_maps):
            for i, j in product(range(f), repeat=2):
                f_k = self.image_size / self.steps[k]
                # unit center x,y
                cx = (j + 0.5) / f_k
                cy = (i + 0.5) / f_k

                # aspect_ratio: 1
                # rel size: min_size
                s_k = self.min_sizes[k]/self.image_size
                mean += [cx, cy, s_k, s_k]

                # aspect_ratio: 1
                # rel size: sqrt(s_k * s_(k+1))
                s_k_prime = sqrt(s_k * (self.max_sizes[k]/self.image_size))
                mean += [cx, cy, s_k_prime, s_k_prime]

                # rest of aspect ratios
                for ar in self.aspect_ratios[k]:
                    mean += [cx, cy, s_k*sqrt(ar), s_k/sqrt(ar)]
                    mean += [cx, cy, s_k/sqrt(ar), s_k*sqrt(ar)]
        # back to torch land
        output = torch.Tensor(mean).view(-1, 4)
        if self.clip:
            output.clamp_(max=1, min=0)
        return output


In [0]:
import torch
import torch.nn as nn
from torch.autograd import Function
from torch.autograd import Variable
import torch.nn.init as init

class L2Norm(nn.Module):
    def __init__(self,n_channels, scale):
        super(L2Norm,self).__init__()
        self.n_channels = n_channels
        self.gamma = scale or None
        self.eps = 1e-10
        self.weight = nn.Parameter(torch.Tensor(self.n_channels))
        self.reset_parameters()

    def reset_parameters(self):
        init.constant_(self.weight,self.gamma)

    def forward(self, x):
        norm = x.pow(2).sum(dim=1, keepdim=True).sqrt()+self.eps
        #x /= norm
        x = torch.div(x,norm)
        out = self.weight.unsqueeze(0).unsqueeze(2).unsqueeze(3).expand_as(x) * x
        return out


In [0]:
import torch.nn.functional as F

class MultiBoxLoss(nn.Module):
    """SSD Weighted Loss Function
    Compute Targets:
        1) Produce Confidence Target Indices by matching  ground truth boxes
           with (default) 'priorboxes' that have jaccard index > threshold parameter
           (default threshold: 0.5).
        2) Produce localization target by 'encoding' variance into offsets of ground
           truth boxes and their matched  'priorboxes'.
        3) Hard negative mining to filter the excessive number of negative examples
           that comes with using a large number of default bounding boxes.
           (default negative:positive ratio 3:1)
    Objective Loss:
        L(x,c,l,g) = (Lconf(x, c) + αLloc(x,l,g)) / N
        Where, Lconf is the CrossEntropy Loss and Lloc is the SmoothL1 Loss
        weighted by α which is set to 1 by cross val.
        Args:
            c: class confidences,
            l: predicted boxes,
            g: ground truth boxes
            N: number of matched default boxes
        See: https://arxiv.org/pdf/1512.02325.pdf for more details.
    """

    def __init__(self, cfg, overlap_thresh, prior_for_matching,
                 bkg_label, neg_mining, neg_pos, neg_overlap, encode_target,
                 use_gpu=True):
        super(MultiBoxLoss, self).__init__()
        self.use_gpu = use_gpu
        self.num_classes = cfg['num_classes']
        self.threshold = overlap_thresh
        self.background_label = bkg_label
        self.encode_target = encode_target
        self.use_prior_for_matching = prior_for_matching
        self.do_neg_mining = neg_mining
        self.negpos_ratio = neg_pos
        self.neg_overlap = neg_overlap
        self.variance = cfg['variance']

    def forward(self, predictions, targets):
        """Multibox Loss
        Args:
            predictions (tuple): A tuple containing loc preds, conf preds,
            and prior boxes from SSD net.
                conf shape: torch.size(batch_size,num_priors,num_classes)
                loc shape: torch.size(batch_size,num_priors,4)
                priors shape: torch.size(num_priors,4)

            targets (tensor): Ground truth boxes and labels for a batch,
                shape: [batch_size,num_objs,5] (last idx is the label).
        """
        loc_data, conf_data, priors = predictions
        num = loc_data.size(0)
        priors = priors[:loc_data.size(1), :]
        num_priors = (priors.size(0))
        num_classes = self.num_classes

        # match priors (default boxes) and ground truth boxes
        loc_t = torch.Tensor(num, num_priors, 4)
        conf_t = torch.LongTensor(num, num_priors)
        for idx in range(num):
            truths = targets[idx][:, :-1].data
            labels = targets[idx][:, -1].data
            defaults = priors.data
            match(self.threshold, truths, defaults, self.variance, labels,
                  loc_t, conf_t, idx)
        if self.use_gpu:
            loc_t = loc_t.cuda()
            conf_t = conf_t.cuda()
        # wrap targets
        loc_t = Variable(loc_t, requires_grad=False)
        conf_t = Variable(conf_t, requires_grad=False)

        pos = conf_t > 0
        num_pos = pos.sum(dim=1, keepdim=True)

        # Localization Loss (Smooth L1)
        # Shape: [batch,num_priors,4]
        pos_idx = pos.unsqueeze(pos.dim()).expand_as(loc_data)
        loc_p = loc_data[pos_idx].view(-1, 4)
        loc_t = loc_t[pos_idx].view(-1, 4)
        # size_average and reduce args will be deprecated, please use reduction='sum' instead.
        # loss_l = F.smooth_l1_loss(loc_p, loc_t, size_average=False)
        loss_l = F.smooth_l1_loss(loc_p, loc_t, reduction='sum')


        # Compute max conf across batch for hard negative mining
        batch_conf = conf_data.view(-1, self.num_classes)
        loss_c = log_sum_exp(batch_conf) - batch_conf.gather(1, conf_t.view(-1, 1))

        # Hard Negative Mining
        # https://github.com/amdegroot/ssd.pytorch/issues/173
        loss_c = loss_c.view(num, -1)
        loss_c[pos] = 0  # filter out pos boxes for now
        _, loss_idx = loss_c.sort(1, descending=True)
        _, idx_rank = loss_idx.sort(1)
        num_pos = pos.long().sum(1, keepdim=True)
        num_neg = torch.clamp(self.negpos_ratio*num_pos, max=pos.size(1)-1)
        neg = idx_rank < num_neg.expand_as(idx_rank)

        # Confidence Loss Including Positive and Negative Examples
        pos_idx = pos.unsqueeze(2).expand_as(conf_data)
        neg_idx = neg.unsqueeze(2).expand_as(conf_data)
        conf_p = conf_data[(pos_idx+neg_idx).gt(0)].view(-1, self.num_classes)
        targets_weighted = conf_t[(pos+neg).gt(0)]
        loss_c = F.cross_entropy(conf_p, targets_weighted, reduction='sum')

        # Sum of losses: L(x,c,l,g) = (Lconf(x, c) + αLloc(x,l,g)) / N
        # https://github.com/amdegroot/ssd.pytorch/issues/173
        N = num_pos.data.sum().double()
        loss_l = loss_l.double()
        loss_c = loss_c.double()
        loss_l /= N
        loss_c /= N
        return loss_l, loss_c


In [0]:
import torch
from torchvision import transforms
import cv2
import numpy as np
import types
from numpy import random


def intersect_numpy(box_a, box_b):
    
    max_xy = np.minimum(box_a[:, 2:], box_b[2:])
    min_xy = np.maximum(box_a[:, :2], box_b[:2])
    inter = np.clip((max_xy - min_xy), a_min=0, a_max=np.inf)
    return inter[:, 0] * inter[:, 1]


def jaccard_numpy(box_a, box_b):
    """Compute the jaccard overlap of two sets of boxes.  The jaccard overlap
    is simply the intersection over union of two boxes.
    E.g.:
        A ∩ B / A ∪ B = A ∩ B / (area(A) + area(B) - A ∩ B)
    Args:
        box_a: Multiple bounding boxes, Shape: [num_boxes,4]
        box_b: Single bounding box, Shape: [4]
    Return:
        jaccard overlap: Shape: [box_a.shape[0], box_a.shape[1]]
    """
    inter = intersect_numpy(box_a, box_b)
    area_a = ((box_a[:, 2]-box_a[:, 0]) *
              (box_a[:, 3]-box_a[:, 1]))  # [A,B]
    area_b = ((box_b[2]-box_b[0]) *
              (box_b[3]-box_b[1]))  # [A,B]
    union = area_a + area_b - inter
    return inter / union  # [A,B]


class Compose(object):
    """Composes several augmentations together.
    Args:
        transforms (List[Transform]): list of transforms to compose.
    Example:
        >>> augmentations.Compose([
        >>>     transforms.CenterCrop(10),
        >>>     transforms.ToTensor(),
        >>> ])
    """

    def __init__(self, transforms):
        self.transforms = transforms

    def __call__(self, img, boxes=None, labels=None):
        for t in self.transforms:
            img, boxes, labels = t(img, boxes, labels)
        return img, boxes, labels


class Lambda(object):
    """Applies a lambda as a transform."""

    def __init__(self, lambd):
        assert isinstance(lambd, types.LambdaType)
        self.lambd = lambd

    def __call__(self, img, boxes=None, labels=None):
        return self.lambd(img, boxes, labels)


class ConvertFromInts(object):
    def __call__(self, image, boxes=None, labels=None):
        return image.astype(np.float32), boxes, labels


class SubtractMeans(object):
    def __init__(self, mean):
        self.mean = np.array(mean, dtype=np.float32)

    def __call__(self, image, boxes=None, labels=None):
        image = image.astype(np.float32)
        image -= self.mean
        return image.astype(np.float32), boxes, labels


class ToAbsoluteCoords(object):
    def __call__(self, image, boxes=None, labels=None):
        height, width, channels = image.shape
        boxes[:, 0] *= width
        boxes[:, 2] *= width
        boxes[:, 1] *= height
        boxes[:, 3] *= height

        return image, boxes, labels


class ToPercentCoords(object):
    def __call__(self, image, boxes=None, labels=None):
        height, width, channels = image.shape
        boxes[:, 0] /= width
        boxes[:, 2] /= width
        boxes[:, 1] /= height
        boxes[:, 3] /= height

        return image, boxes, labels


class Resize(object):
    def __init__(self, size=300):
        self.size = size

    def __call__(self, image, boxes=None, labels=None):
        image = cv2.resize(image, (self.size,
                                 self.size))
        return image, boxes, labels


class RandomSaturation(object):
    def __init__(self, lower=0.5, upper=1.5):
        self.lower = lower
        self.upper = upper
        assert self.upper >= self.lower, "contrast upper must be >= lower."
        assert self.lower >= 0, "contrast lower must be non-negative."

    def __call__(self, image, boxes=None, labels=None):
        if random.randint(2):
            image[:, :, 1] *= random.uniform(self.lower, self.upper)

        return image, boxes, labels


class RandomHue(object):
    def __init__(self, delta=18.0):
        assert delta >= 0.0 and delta <= 360.0
        self.delta = delta

    def __call__(self, image, boxes=None, labels=None):
        if random.randint(2):
            image[:, :, 0] += random.uniform(-self.delta, self.delta)
            image[:, :, 0][image[:, :, 0] > 360.0] -= 360.0
            image[:, :, 0][image[:, :, 0] < 0.0] += 360.0
        return image, boxes, labels


class RandomLightingNoise(object):
    def __init__(self):
        self.perms = ((0, 1, 2), (0, 2, 1),
                      (1, 0, 2), (1, 2, 0),
                      (2, 0, 1), (2, 1, 0))

    def __call__(self, image, boxes=None, labels=None):
        if random.randint(2):
            swap = self.perms[random.randint(len(self.perms))]
            shuffle = SwapChannels(swap)  # shuffle channels
            image = shuffle(image)
        return image, boxes, labels


class ConvertColor(object):
    def __init__(self, current='BGR', transform='HSV'):
        self.transform = transform
        self.current = current

    def __call__(self, image, boxes=None, labels=None):
        if self.current == 'BGR' and self.transform == 'HSV':
            image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        elif self.current == 'HSV' and self.transform == 'BGR':
            image = cv2.cvtColor(image, cv2.COLOR_HSV2BGR)
        else:
            raise NotImplementedError
        return image, boxes, labels


class RandomContrast(object):
    def __init__(self, lower=0.5, upper=1.5):
        self.lower = lower
        self.upper = upper
        assert self.upper >= self.lower, "contrast upper must be >= lower."
        assert self.lower >= 0, "contrast lower must be non-negative."

    # expects float image
    def __call__(self, image, boxes=None, labels=None):
        if random.randint(2):
            alpha = random.uniform(self.lower, self.upper)
            image *= alpha
        return image, boxes, labels


class RandomBrightness(object):
    def __init__(self, delta=32):
        assert delta >= 0.0
        assert delta <= 255.0
        self.delta = delta

    def __call__(self, image, boxes=None, labels=None):
        if random.randint(2):
            delta = random.uniform(-self.delta, self.delta)
            image += delta
        return image, boxes, labels


class ToCV2Image(object):
    def __call__(self, tensor, boxes=None, labels=None):
        return tensor.cpu().numpy().astype(np.float32).transpose((1, 2, 0)), boxes, labels


class ToTensor(object):
    def __call__(self, cvimage, boxes=None, labels=None):
        return torch.from_numpy(cvimage.astype(np.float32)).permute(2, 0, 1), boxes, labels


class RandomSampleCrop(object):
    """Crop
    Arguments:
        img (Image): the image being input during training
        boxes (Tensor): the original bounding boxes in pt form
        labels (Tensor): the class labels for each bbox
        mode (float tuple): the min and max jaccard overlaps
    Return:
        (img, boxes, classes)
            img (Image): the cropped image
            boxes (Tensor): the adjusted bounding boxes in pt form
            labels (Tensor): the class labels for each bbox
    """
    def __init__(self):
        self.sample_options = (
            # using entire original input image
            None,
            # sample a patch s.t. MIN jaccard w/ obj in .1,.3,.4,.7,.9
            (0.1, None),
            (0.3, None),
            (0.7, None),
            (0.9, None),
            # randomly sample a patch
            (None, None),
        )

    def __call__(self, image, boxes=None, labels=None):
        height, width, _ = image.shape
        while True:
            # randomly choose a mode
            mode = random.choice(self.sample_options)
            if mode is None:
                return image, boxes, labels

            min_iou, max_iou = mode
            if min_iou is None:
                min_iou = float('-inf')
            if max_iou is None:
                max_iou = float('inf')

            # max trails (50)
            for _ in range(50):
                current_image = image

                w = random.uniform(0.3 * width, width)
                h = random.uniform(0.3 * height, height)

                # aspect ratio constraint b/t .5 & 2
                if h / w < 0.5 or h / w > 2:
                    continue

                left = random.uniform(width - w)
                top = random.uniform(height - h)

                # convert to integer rect x1,y1,x2,y2
                rect = np.array([int(left), int(top), int(left+w), int(top+h)])

                # calculate IoU (jaccard overlap) b/t the cropped and gt boxes
                overlap = jaccard_numpy(boxes, rect)

                # is min and max overlap constraint satisfied? if not try again
                if overlap.min() < min_iou and max_iou < overlap.max():
                    continue

                # cut the crop from the image
                current_image = current_image[rect[1]:rect[3], rect[0]:rect[2],
                                              :]

                # keep overlap with gt box IF center in sampled patch
                centers = (boxes[:, :2] + boxes[:, 2:]) / 2.0

                # mask in all gt boxes that above and to the left of centers
                m1 = (rect[0] < centers[:, 0]) * (rect[1] < centers[:, 1])

                # mask in all gt boxes that under and to the right of centers
                m2 = (rect[2] > centers[:, 0]) * (rect[3] > centers[:, 1])

                # mask in that both m1 and m2 are true
                mask = m1 * m2

                # have any valid boxes? try again if not
                if not mask.any():
                    continue

                # take only matching gt boxes
                current_boxes = boxes[mask, :].copy()

                # take only matching gt labels
                current_labels = labels[mask]

                # should we use the box left and top corner or the crop's
                current_boxes[:, :2] = np.maximum(current_boxes[:, :2],
                                                  rect[:2])
                # adjust to crop (by substracting crop's left,top)
                current_boxes[:, :2] -= rect[:2]

                current_boxes[:, 2:] = np.minimum(current_boxes[:, 2:],
                                                  rect[2:])
                # adjust to crop (by substracting crop's left,top)
                current_boxes[:, 2:] -= rect[:2]

                return current_image, current_boxes, current_labels


class Expand(object):
    def __init__(self, mean):
        self.mean = mean

    def __call__(self, image, boxes, labels):
        if random.randint(2):
            return image, boxes, labels

        height, width, depth = image.shape
        ratio = random.uniform(1, 4)
        left = random.uniform(0, width*ratio - width)
        top = random.uniform(0, height*ratio - height)

        expand_image = np.zeros(
            (int(height*ratio), int(width*ratio), depth),
            dtype=image.dtype)
        expand_image[:, :, :] = self.mean
        expand_image[int(top):int(top + height),
                     int(left):int(left + width)] = image
        image = expand_image

        boxes = boxes.copy()
        boxes[:, :2] += (int(left), int(top))
        boxes[:, 2:] += (int(left), int(top))

        return image, boxes, labels


class RandomMirror(object):
    def __call__(self, image, boxes, classes):
        _, width, _ = image.shape
        if random.randint(2):
            image = image[:, ::-1]
            boxes = boxes.copy()
            boxes[:, 0::2] = width - boxes[:, 2::-2]
        return image, boxes, classes


class SwapChannels(object):
    """Transforms a tensorized image by swapping the channels in the order
     specified in the swap tuple.
    Args:
        swaps (int triple): final order of channels
            eg: (2, 1, 0)
    """

    def __init__(self, swaps):
        self.swaps = swaps

    def __call__(self, image):
        """
        Args:
            image (Tensor): image tensor to be transformed
        Return:
            a tensor with channels swapped according to swap
        """
        # if torch.is_tensor(image):
        #     image = image.data.cpu().numpy()
        # else:
        #     image = np.array(image)
        image = image[:, :, self.swaps]
        return image


class PhotometricDistort(object):
    def __init__(self):
        self.pd = [
            RandomContrast(),
            ConvertColor(transform='HSV'),
            RandomSaturation(),
            RandomHue(),
            ConvertColor(current='HSV', transform='BGR'),
            RandomContrast()
        ]
        self.rand_brightness = RandomBrightness()
        self.rand_light_noise = RandomLightingNoise()

    def __call__(self, image, boxes, labels):
        im = image.copy()
        im, boxes, labels = self.rand_brightness(im, boxes, labels)
        if random.randint(2):
            distort = Compose(self.pd[:-1])
        else:
            distort = Compose(self.pd[1:])
        im, boxes, labels = distort(im, boxes, labels)
        return self.rand_light_noise(im, boxes, labels)


class SSDAugmentation(object):
    def __init__(self, size=300, mean=(104, 117, 123)):
        self.mean = mean
        self.size = size
        self.augment = Compose([
            ConvertFromInts(),
            ToAbsoluteCoords(),
            PhotometricDistort(),
            Expand(self.mean),
            RandomSampleCrop(),
            RandomMirror(),
            ToPercentCoords(),
            Resize(self.size),
            SubtractMeans(self.mean)
        ])

    def __call__(self, img, boxes, labels):
        return self.augment(img, boxes, labels)


In [0]:

class SSD(nn.Module):
    """Single Shot Multibox Architecture
    The network is composed of a base VGG network followed by the
    added multibox conv layers.  Each multibox layer branches into
        1) conv2d for class conf scores
        2) conv2d for localization predictions
        3) associated priorbox layer to produce default bounding
           boxes specific to the layer's feature map size.
    See: https://arxiv.org/pdf/1512.02325.pdf for more details.

    Args:
        phase: (string) Can be "test" or "train"
        size: input image size
        base: VGG16 layers for input, size of either 300 or 500
        extras: extra layers that feed to multibox loc and conf layers
        head: "multibox head" consists of loc and conf conv layers
    """

    def __init__(self, phase, size, base, extras, head, num_classes):
        super(SSD, self).__init__()
        self.phase = phase
        self.num_classes = num_classes
        self.cfg = _sixray
        self.priorbox = PriorBox(self.cfg)
        with torch.no_grad():
          self.priors = Variable(self.priorbox.forward())
        self.size = size

        # SSD network
        self.vgg = nn.ModuleList(base)
        # Layer learns to scale the l2 normalized features from conv4_3
        self.L2Norm = L2Norm(512, 20)
        self.extras = nn.ModuleList(extras)

        self.loc = nn.ModuleList(head[0])
        self.conf = nn.ModuleList(head[1])

        if phase == 'test':
            self.softmax = nn.Softmax(dim=-1)
            self.detect = Detect(num_classes, 0, 200, 0.01, 0.45)

    def forward(self, x):
        """Applies network layers and ops on input image(s) x.

        Args:
            x: input image or batch of images. Shape: [batch,3,300,300].

        Return:
            Depending on phase:
            test:
                Variable(tensor) of output class label predictions,
                confidence score, and corresponding location predictions for
                each object detected. Shape: [batch,topk,7]

            train:
                list of concat outputs from:
                    1: confidence layers, Shape: [batch*num_priors,num_classes]
                    2: localization layers, Shape: [batch,num_priors*4]
                    3: priorbox layers, Shape: [2,num_priors*4]
        """
        sources = list()
        loc = list()
        conf = list()

        # apply vgg up to conv4_3 relu
        for k in range(23):
            x = self.vgg[k](x)

        s = self.L2Norm(x)
        sources.append(s)

        # apply vgg up to fc7
        for k in range(23, len(self.vgg)):
            x = self.vgg[k](x)
        sources.append(x)

        # apply extra layers and cache source layer outputs
        for k, v in enumerate(self.extras):
            x = F.relu(v(x), inplace=True)
            if k % 2 == 1:
                sources.append(x)

        # apply multibox head to source layers
        for (x, l, c) in zip(sources, self.loc, self.conf):
            loc.append(l(x).permute(0, 2, 3, 1).contiguous())
            conf.append(c(x).permute(0, 2, 3, 1).contiguous())

        loc = torch.cat([o.view(o.size(0), -1) for o in loc], 1)
        conf = torch.cat([o.view(o.size(0), -1) for o in conf], 1)
        if self.phase == "test":
            output = self.detect(
                loc.view(loc.size(0), -1, 4),                   # loc preds
                self.softmax(conf.view(conf.size(0), -1,
                             self.num_classes)),                # conf preds
                self.priors.type(type(x.data))                  # default boxes
            )
        else:
            output = (
                loc.view(loc.size(0), -1, 4),
                conf.view(conf.size(0), -1, self.num_classes),
                self.priors
            )
        return output

    def load_weights(self, base_file):
        other, ext = os.path.splitext(base_file)
        if ext == '.pkl' or '.pth':
            print('Loading weights into state dict...')
            self.load_state_dict(torch.load(base_file,
                                 map_location=lambda storage, loc: storage))
            print('Finished!')
        else:
            print('Sorry only .pth and .pkl files supported.')


# This function is derived from torchvision VGG make_layers()
# https://github.com/pytorch/vision/blob/master/torchvision/models/vgg.py
def vgg(cfg, i, batch_norm=False):
    layers = []
    in_channels = i
    for v in cfg:
        if v == 'M':
            layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
        elif v == 'C':
            layers += [nn.MaxPool2d(kernel_size=2, stride=2, ceil_mode=True)]
        else:
            conv2d = nn.Conv2d(in_channels, v, kernel_size=3, padding=1)
            if batch_norm:
                layers += [conv2d, nn.BatchNorm2d(v), nn.ReLU(inplace=True)]
            else:
                layers += [conv2d, nn.ReLU(inplace=True)]
            in_channels = v
    pool5 = nn.MaxPool2d(kernel_size=3, stride=1, padding=1)
    conv6 = nn.Conv2d(512, 1024, kernel_size=3, padding=6, dilation=6)
    conv7 = nn.Conv2d(1024, 1024, kernel_size=1)
    layers += [pool5, conv6,
               nn.ReLU(inplace=True), conv7, nn.ReLU(inplace=True)]
    return layers


def add_extras(cfg, i, batch_norm=False):
    # Extra layers added to VGG for feature scaling
    layers = []
    in_channels = i
    flag = False
    for k, v in enumerate(cfg):
        if in_channels != 'S':
            if v == 'S':
                layers += [nn.Conv2d(in_channels, cfg[k + 1],
                           kernel_size=(1, 3)[flag], stride=2, padding=1)]
            else:
                layers += [nn.Conv2d(in_channels, v, kernel_size=(1, 3)[flag])]
            flag = not flag
        in_channels = v
    return layers


def multibox(vgg, extra_layers, cfg, num_classes):
    loc_layers = []
    conf_layers = []
    vgg_source = [21, -2]
    for k, v in enumerate(vgg_source):
        loc_layers += [nn.Conv2d(vgg[v].out_channels,
                                 cfg[k] * 4, kernel_size=3, padding=1)]
        conf_layers += [nn.Conv2d(vgg[v].out_channels,
                        cfg[k] * num_classes, kernel_size=3, padding=1)]
    for k, v in enumerate(extra_layers[1::2], 2):
        loc_layers += [nn.Conv2d(v.out_channels, cfg[k]
                                 * 4, kernel_size=3, padding=1)]
        conf_layers += [nn.Conv2d(v.out_channels, cfg[k]
                                  * num_classes, kernel_size=3, padding=1)]
    return vgg, extra_layers, (loc_layers, conf_layers)


base = {
    '300': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'C', 512, 512, 512, 'M',
            512, 512, 512],
    '512': [],
}
extras = {
    '300': [256, 'S', 512, 128, 'S', 256, 128, 256, 128, 256],
    '512': [],
}
mbox = {
    '300': [4, 6, 6, 6, 4, 4],  # number of boxes per feature map location
    '512': [],
}


def build_ssd(phase, size=300, num_classes=21):
    if phase != "test" and phase != "train":
        print("ERROR: Phase: " + phase + " not recognized")
        return
    if size != 300:
        print("ERROR: You specified size " + repr(size) + ". However, " +
              "currently only SSD300 (size=300) is supported!")
        return
    base_, extras_, head_ = multibox(vgg(base[str(size)], 3),
                                     add_extras(extras[str(size)], 1024),
                                     mbox[str(size)], num_classes)
    return SSD(phase, size, base_, extras_, head_, num_classes)


In [0]:
import time
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
import torch.utils.data as data
import argparse


def str2bool(v):
    return v.lower() in ("yes", "true", "t", "1")


parser = argparse.ArgumentParser(
    description='Single Shot MultiBox Detector Training With Pytorch')
train_set = parser.add_mutually_exclusive_group()
parser.add_argument('--dataset', default='VOC', choices=['VOC', 'COCO'],
                    type=str, help='VOC or COCO')
parser.add_argument('--dataset_root', default='/content/gdrive/My Drive/weights/',
                    help='Dataset root directory path')
parser.add_argument('--basenet', default='VOC.pth',
                    help='Pretrained base model')
parser.add_argument('--batch_size', default=30, type=int,
                    help='Batch size for training')
parser.add_argument('--resume', default=None, type=str,
                    help='Checkpoint state_dict file to resume training from')
parser.add_argument('--start_iter', default=0, type=int,
                    help='Resume training at this iter')
parser.add_argument('--num_workers', default=4, type=int,
                    help='Number of workers used in dataloading')
parser.add_argument('--cuda', default=True, type=str2bool,
                    help='Use CUDA to train model')
parser.add_argument('--lr', '--learning-rate', default=0.0002, type=float,
                    help='initial learning rate')
parser.add_argument('--momentum', default=0.9, type=float,
                    help='Momentum value for optim')
parser.add_argument('--weight_decay', default=5e-4, type=float,
                    help='Weight decay for SGD')
parser.add_argument('--gamma', default=0.1, type=float,
                    help='Gamma update for SGD')
parser.add_argument('--visdom', default=False, type=str2bool,
                    help='Use visdom for loss visualization')
parser.add_argument('--save_folder', default='weights/',
                    help='Directory for saving checkpoint models')
args = parser.parse_args([])


if torch.cuda.is_available():
    if args.cuda:
        torch.set_default_tensor_type('torch.cuda.FloatTensor')
    if not args.cuda:
        print("WARNING: It looks like you have a CUDA device, but aren't " +
              "using CUDA.\nRun with --cuda for optimal training speed.")
        torch.set_default_tensor_type('torch.FloatTensor')
else:
    torch.set_default_tensor_type('torch.FloatTensor')

if not os.path.exists(args.save_folder):
    os.mkdir(args.save_folder)

# import visdom
# viz = visdom.Visdom()
# python -m visdom.server

def train():
    # if args.dataset == 'COCO':
    #     if args.dataset_root == VOC_ROOT:
    #         if not os.path.exists(COCO_ROOT):
    #             parser.error('Must specify dataset_root if specifying dataset')
    #         print("WARNING: Using default COCO dataset_root because " +
    #               "--dataset_root was not specified.")
    #         args.dataset_root = COCO_ROOT
    #     cfg = coco
    #     dataset = COCODetection(root=args.dataset_root,
    #                             transform=SSDAugmentation(cfg['min_dim'],
    #                                                       MEANS))
    # elif args.dataset == 'VOC':
    #     if args.dataset_root == COCO_ROOT:
    #         parser.error('Must specify dataset if specifying dataset_root')
    #     cfg = voc
    #     dataset = VOCDetection(root=args.dataset_root,
    #                            transform=SSDAugmentation(cfg['min_dim'],
    #                                                      MEANS))

    cfg = _sixray
    cfg['num_classes'] = 3

    dataset_mean = (104, 117, 123)
    dataset = SIXrayDetection("/content/gdrive/My Drive/all/", "",
                              # BaseTransform(cfg['min_dim'], MEANS),
                              SSDAugmentation(cfg['min_dim'], MEANS),
                              SIXrayAnnotationTransform())

    # if args.visdom:

    ssd_net = build_ssd('train', cfg['min_dim'], cfg['num_classes'])
    net = ssd_net

    if args.cuda:
        net = torch.nn.DataParallel(ssd_net)
        cudnn.benchmark = True

    if args.resume:
        print('Resuming training, loading {}...'.format(args.resume))
        ssd_net.load_weights(args.resume)
    else:
        vgg_weights = torch.load(args.save_folder + args.basenet)
        # from collections import OrderedDict
        # new_state_dict = OrderedDict()
        # for k, v in vgg_weights.items():
        #     name = k[4:] # remove 'module.' of dataparallel
        #     new_state_dict[name]=v
        # print('Loading base network...')
        ssd_net.load_state_dict(vgg_weights)
        # ssd_net.vgg.load_state_dict(vgg_weights)
    
    if args.cuda:
        net = net.cuda()

    if not args.resume:
        print('Initializing weights...')
        # initialize newly added layers' weights with xavier method
        ssd_net.extras.apply(weights_init)
        ssd_net.loc.apply(weights_init)
        ssd_net.conf.apply(weights_init)

    optimizer = optim.SGD(net.parameters(), lr=args.lr, momentum=args.momentum,
                          weight_decay=args.weight_decay)
    criterion = MultiBoxLoss(cfg, 0.5, True, 0, True, 3, 0.5,
                             False, args.cuda)

    net.train()
    # loss counters
    loc_loss = 0
    conf_loss = 0
    epoch = 0
    print('Loading the dataset...')

    epoch_size = len(dataset) // args.batch_size
    print('Training SSD on:', dataset.name)
    print('Using the specified args:')
    print(args)

    step_index = 0

    # if args.visdom:
    #     vis_title = 'SSD.PyTorch on ' + dataset.name
    #     vis_legend = ['Loc Loss', 'Conf Loss', 'Total Loss']
    #     iter_plot = create_vis_plot('Iteration', 'Loss', vis_title, vis_legend)
    #     epoch_plot = create_vis_plot('Epoch', 'Loss', vis_title, vis_legend)
    i = 0
    for iteration in range(args.start_iter,26):
        data_loader = data.DataLoader(dataset, 20,
                                      num_workers=args.num_workers,
                                      shuffle=True, collate_fn=detection_collate,
                                      pin_memory=True)
        # create batch iterator
        batch_iterator = iter(data_loader)
        # if args.visdom and iteration != 0 and (iteration % epoch_size == 0):
        #     update_vis_plot(epoch, loc_loss, conf_loss, epoch_plot, None,
        #                     'append', epoch_size)
        #     # reset epoch loss counters
        #     loc_loss = 0
        #     conf_loss = 0
        #     epoch += 1
        # if i in cfg['lr_steps']:
        # if iteration in [1, 2, 4]:
        # if iteration == 19 or iteration == 20 or iteration == 30: 
        #   step_index += 1
        #   adjust_learning_rate(optimizer, args.gamma, step_index)
        if iteration in cfg['lr_steps']:
            step_index += 1
            adjust_learning_rate(optimizer, args.gamma, step_index)
        for j in range(240):
          i += 1
          # load train data
          images, targets = next(batch_iterator)

          if args.cuda:
              images = Variable(images.cuda())
              with torch.no_grad():
                targets = [Variable(ann.cuda()) for ann in targets]
          else:
            images = Variable(images)
            with torch.no_grad():
              targets = [Variable(ann) for ann in targets]
          # forward
          t0 = time.time()
          out = net(images)
          # backprop
          optimizer.zero_grad()
          # print(type(out))
          # print(out)
          # print(type(targets))
          loss_l, loss_c = criterion(out, targets)
          loss = loss_l + loss_c
          loss.backward()
          optimizer.step()
          t1 = time.time()
          # https://github.com/NVIDIA/flownet2-pytorch/issues/113
          loc_loss += loss_l.data
          conf_loss += loss_c.data
        # if iteration % 10 == 0:
        #     print('timer: %.4f sec.' % (t1 - t0))
        #     print('iter ' + repr(iteration) + ' || Loss: %.4f ||' % (loss.data[0]), end=' ')

        # if args.visdom:
        #     update_vis_plot(iteration, loss_l.data[0], loss_c.data[0],
        #                     iter_plot, epoch_plot, 'append')

        # if iteration != 0 and iteration % 5000 == 0:
        #     print('Saving state, iter:', iteration)
        #     torch.save(ssd_net.state_dict(), 'weights/ssd300_COCO_' +
        #                repr(iteration) + '.pth')
          if (i%1000 == 0):
              print('iter ' + repr(i) + ' || Loss: %.4f || loss_l: %4f || loss_c: %4f \n' % (loss.data, loss_l.data, loss_c.data))
              torch.save(ssd_net.state_dict(),
                        args.save_folder + '' + 'sixray_9_1_' + str(i) + '.pth')
    # 3, 19200


def adjust_learning_rate(optimizer, gamma, step):
    """Sets the learning rate to the initial LR decayed by 10 at every
        specified step
    # Adapted from PyTorch Imagenet example:
    # https://github.com/pytorch/examples/blob/master/imagenet/main.py
    """
    lr = args.lr * (gamma ** (step))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr


def xavier(param):
    init.xavier_uniform_(param)


def weights_init(m):
    if isinstance(m, nn.Conv2d):
        xavier(m.weight.data)
        m.bias.data.zero_()


# def create_vis_plot(_xlabel, _ylabel, _title, _legend):
#     return viz.line(
#         X=torch.zeros((1,)).cpu(),
#         Y=torch.zeros((1, 3)).cpu(),
#         opts=dict(
#             xlabel=_xlabel,
#             ylabel=_ylabel,
#             title=_title,
#             legend=_legend
#         )
#     )


# def update_vis_plot(iteration, loc, conf, window1, window2, update_type,
#                     epoch_size=1):
#     viz.line(
#         X=torch.ones((1, 3)).cpu() * iteration,
#         Y=torch.Tensor([loc, conf, loc + conf]).unsqueeze(0).cpu() / epoch_size,
#         win=window1,
#         update=update_type
#     )
#     # initialize epoch plot on first iteration
#     if iteration == 0:
#         viz.line(
#             X=torch.zeros((1, 3)).cpu(),
#             Y=torch.Tensor([loc, conf, loc + conf]).unsqueeze(0).cpu(),
#             win=window2,
#             update=True
#         )


if __name__ == '__main__':
    train()


['coreless_battery00004025.txt', 'coreless_battery00004041.txt', 'coreless_battery00004057.txt', 'coreless_battery00004083.txt', 'coreless_battery00004106.txt', 'coreless_battery00004114.txt', 'coreless_battery00004133.txt', 'coreless_battery00004172.txt', 'coreless_battery00004179.txt', 'coreless_battery00004188.txt', 'coreless_battery00004234.txt', 'coreless_battery00004242.txt', 'coreless_battery00004255.txt', 'coreless_battery00004289.txt', 'coreless_battery00004127.txt', 'coreless_battery00004307.txt', 'coreless_battery00004327.txt', 'coreless_battery00004336.txt', 'coreless_battery00004366.txt', 'coreless_battery00004383.txt', 'coreless_battery00004411.txt', 'coreless_battery00004428.txt', 'coreless_battery00004461.txt', 'coreless_battery00004466.txt', 'coreless_battery00004514.txt', 'coreless_battery00004528.txt', 'coreless_battery00004534.txt', 'coreless_battery00004566.txt', 'coreless_battery00004600.txt', 'coreless_battery00004609.txt', 'coreless_battery00004621.txt', 'corele

In [0]:
print(1001 % 1000)

In [0]:
# """Adapted from:
#     @longcw faster_rcnn_pytorch: https://github.com/longcw/faster_rcnn_pytorch
#     @rbgirshick py-faster-rcnn https://github.com/rbgirshick/py-faster-rcnn
#     Licensed under The MIT License [see LICENSE for details]
# """

# from __future__ import print_function
# import torch
# import torch.nn as nn
# import torch.backends.cudnn as cudnn
# from torch.autograd import Variable
# import torch.utils.data as data
# import sys
# import os
# import os.path as osp
# import time
# import argparse
# import numpy as np
# import pickle
# import cv2
# import shutil

# if sys.version_info[0] == 2:
#     import xml.etree.cElementTree as ET
# else:
#     import xml.etree.ElementTree as ET

# labelmap = SIXray_CLASSES


# def str2bool(v):
#     return v.lower() in ("yes", "true", "t", "a1")


# EPOCH = 5
# GPUID = '3'
# os.environ["CUDA_VISIBLE_DEVICES"] = GPUID

# parser = argparse.ArgumentParser(
#     description='Single Shot MultiBox Detector Evaluation')
# parser.add_argument('--trained_model',
#                     default="", type=str,
#                     help='Trained state_dict file path to open')
# parser.add_argument(  # '--save_folder', default='/media/dsg3/husheng/eval/', type=str,
#     '--save_folder',
#     default="", type=str,
#     help='File path to save results')
# parser.add_argument('--confidence_threshold', default=0.2, type=float,
#                     help='Detection confidence threshold')
# parser.add_argument('--top_k', default=5, type=int,
#                     help='Further restrict the number of predictions to parse')
# parser.add_argument('--cuda', default=True, type=str2bool,
#                     help='Use cuda to train model')
# parser.add_argument('--SIXray_root', default=SIXray_ROOT,
#                     help='Location of VOC root directory')
# parser.add_argument('--cleanup', default=True, type=str2bool,
#                     help='Cleanup and remove results files following eval')
# parser.add_argument('--imagesetfile',
#                     # default='/media/dsg3/datasets/SIXray/dataset-test.txt', type=str,
#                     default="/media/trs2/Xray20190723/train_test_txt/battery_sub/sub_test_core_coreless.txt", type=str,
#                     help='imageset file path to open')

# args = parser.parse_args([])

# if torch.cuda.is_available():
#     if args.cuda:
#         torch.set_default_tensor_type('torch.cuda.FloatTensor')
#     if not args.cuda:
#         # print("WARNING: It looks like you have a CUDA device, but aren't using \
#         #         CUDA.  Run with --cuda for optimal eval speed.")
#         torch.set_default_tensor_type('torch.FloatTensor')
# else:
#     torch.set_default_tensor_type('torch.FloatTensor')

# annopath = os.path.join(args.SIXray_root, 'Annotation', '%s.txt')
# imgpath = os.path.join(args.SIXray_root, 'Image', '%s.jpg')

# # imgsetpath = os.path.join(args.voc_root, 'VOC2007', 'ImageSets', 'Main', '{:s}.txt')

# YEAR = '2007'

# devkit_path = args.save_folder
# dataset_mean = (104, 117, 123)
# set_type = 'test'


# class Timer(object):
#     """A simple timer."""

#     def __init__(self):
#         self.total_time = 0.
#         self.calls = 0
#         self.start_time = 0.
#         self.diff = 0.
#         self.average_time = 0.

#     def tic(self):
#         # using time.time instead of time.clock because time time.clock
#         # does not normalize for multithreading
#         self.start_time = time.time()

#     def toc(self, average=True):
#         self.diff = time.time() - self.start_time
#         self.total_time += self.diff
#         self.calls += 1
#         self.average_time = self.total_time / self.calls
#         if average:
#             return self.average_time
#         else:
#             return self.diff


# def parse_rec(filename,imgpath):
#     """ Parse a PASCAL VOC xml file """
#     # tree = ET.parse(filename)
#     # filename = filename[:-3] + 'txt'

#     filename = filename.replace('.xml', '.txt')
    
#     #imagename0 = filename.replace('Anno_core_coreless_battery_sub_2000_500', 'cut_Image_core_coreless_battery_sub_2000_500')
#     img_fold_name = imgpath.split('/')[-2]
#     imagename0 = filename.replace('Annotation', img_fold_name)
#     imagename1 = imagename0.replace('.txt', '.jpg')  # jpg form
#     imagename2 = imagename0.replace('.txt', '.jpg')
#     objects = []
#     # 还需要同时打开图像，读入图像大小
#     # print(imagename1)
#     img = cv2.imread(imagename1)
#     if img is None:
#         img = cv2.imread(imagename2)
#     height, width, channels = img.shape
#     with open(filename, "r", encoding='utf-8') as f1:
#         dataread = f1.readlines()
#         for annotation in dataread:
#             obj_struct = {}
#             temp = annotation.split()
#             name = temp[1]
#             if name != '带电芯充电宝' and name != '不带电芯充电宝':
#                 continue
#             xmin = int(temp[2])
#             # 只读取V视角的
#             if int(xmin) > width:
#                 continue
#             if xmin < 0:
#                 xmin = 1
#             ymin = int(temp[3])
#             if ymin < 0:
#                 ymin = 1
#             xmax = int(temp[4])
#             if xmax > width: 
#                 xmax = width - 1
#             ymax = int(temp[5])
#             if ymax > height:
#                 ymax = height - 1
#             ##name
#             obj_struct['name'] = name
#             obj_struct['pose'] = 'Unspecified'
#             obj_struct['truncated'] = 0
#             obj_struct['difficult'] = 0
#             obj_struct['bbox'] = [float(xmin) - 1,
#                                   float(ymin) - 1,
#                                   float(xmax) - 1,
#                                   float(ymax) - 1]
#             objects.append(obj_struct)

#     '''
#     for obj in tree.findall('object'):
#         obj_struct = {}
#         obj_struct['name'] = obj.find('name').text.lower().strip()
#         obj_struct['pose'] = obj.find('pose').text
#         obj_struct['truncated'] = int(obj.find('truncated').text)
#         obj_struct['difficult'] = int(obj.find('difficult').text)
#         bbox = obj.find('bndbox')
#         obj_struct['bbox'] = [float(bbox.find('xmin').text) - 1,
#                               float(bbox.find('ymin').text) - 1,
#                               float(bbox.find('xmax').text) - 1,
#                               float(bbox.find('ymax').text) - 1]
#         objects.append(obj_struct)
#     '''
#     return objects


# def get_output_dir(name, phase):
#     """Return the directory where experimental artifacts are placed.
#     If the directory does not exist, it is created.
#     A canonical path is built using the name from an imdb and a network
#     (if not None).
#     """
#     filedir = os.path.join(name, phase)
#     if not os.path.exists(filedir):
#         os.makedirs(filedir)
#     return filedir


# def get_voc_results_file_template(image_set, cls):
#     # VOCdevkit/VOC2007/results/det_test_aeroplane.txt
#     filename = 'det_' + image_set + '_%s.txt' % (cls)
#     filedir = os.path.join(devkit_path, 'results')
#     if not os.path.exists(filedir):
#         os.makedirs(filedir)
#     path = os.path.join(filedir, filename)
#     return path


# def write_voc_results_file(all_boxes, dataset):
#     for cls_ind, cls in enumerate(labelmap):
#         # print('Writing {:s} VOC results file'.format(cls))
#         filename = get_voc_results_file_template(set_type, cls)
#         with open(filename, 'wt') as f:
#             for im_ind, index in enumerate(dataset.ids):
#                 dets = all_boxes[cls_ind + 1][im_ind]
#                 if dets == []:
#                     continue
#                 # the VOCdevkit expects a1-based indices
#                 for k in range(dets.shape[0]):
#                     f.write('{:s} {:.3f} {:.1f} {:.1f} {:.1f} {:.1f}\n'.
#                             format(index, dets[k, -1],
#                                    dets[k, 0] + 1, dets[k, 1] + 1,
#                                    dets[k, 2] + 1, dets[k, 3] + 1))


# def do_python_eval(output_dir='output', use_07=False):
#     cachedir = os.path.join(devkit_path, 'annotations_cache')
#     aps = []
#     # The PASCAL VOC metric changed in 2010
#     use_07_metric = use_07
#     # print('VOC07 metric? ' + ('Yes' if use_07_metric else 'No'))
#     if not os.path.isdir(output_dir):
#         os.mkdir(output_dir)
#     for i, cls in enumerate(labelmap):
#         filename = get_voc_results_file_template(set_type, cls)
#         rec, prec, ap = voc_eval(
#             filename, annopath,imgpath, args.imagesetfile, cls, cachedir,
#             ovthresh=0.5, use_07_metric=use_07_metric)
#         aps += [ap]
#         # print('AP for {} = {:.4f}'.format(cls, ap))
#         with open(os.path.join(output_dir, cls + '_pr.pkl'), 'wb') as f:
#             pickle.dump({'rec': rec, 'prec': prec, 'ap': ap}, f)
#     print("EPOCH, {:d}, mAP, {:.4f}, core_AP, {:.4f}, coreless_AP, {:.4f}".format(EPOCH, np.mean(aps), aps[0], aps[1]))
#     # print('Mean AP = {:.4f}'.format(np.mean(aps)))
#     '''
#     print('~~~~~~~~')
#     print('Results:')
#     for ap in aps:
#         print('{:.3f}'.format(ap))
#     print('{:.3f}'.format(np.mean(aps)))
#     print('~~~~~~~~')
#     print('--------------------------------------------------------------')
#     print('Results computed with the **unofficial** Python eval code.')
#     print('Results should be very close to the official MATLAB eval code.')
#     print('--------------------------------------------------------------')
#     '''


# def voc_ap(rec, prec, use_07_metric=True):
#     """ ap = voc_ap(rec, prec, [use_07_metric])
#     Compute VOC AP given precision and recall.
#     If use_07_metric is true, uses the
#     VOC 07 11 point method (default:True).
#     """
#     if use_07_metric:
#         # 11 point metric
#         ap = 0.
#         for t in np.arange(0., 1.1, 0.1):
#             if np.sum(rec >= t) == 0:
#                 p = 0
#             else:
#                 p = np.max(prec[rec >= t])
#             ap = ap + p / 11.
#     else:
#         # correct AP calculation
#         # first append sentinel values at the end
#         mrec = np.concatenate(([0.], rec, [1.]))
#         mpre = np.concatenate(([0.], prec, [0.]))

#         # compute the precision envelope
#         for i in range(mpre.size - 1, 0, -1):
#             mpre[i - 1] = np.maximum(mpre[i - 1], mpre[i])

#         # to calculate area under PR curve, look for points
#         # where X axis (recall) changes value
#         i = np.where(mrec[1:] != mrec[:-1])[0]

#         # and sum (\Delta recall) * prec
#         ap = np.sum((mrec[i + 1] - mrec[i]) * mpre[i + 1])
#     return ap


# def voc_eval(detpath,
#              annopath,
# 			       imgpath,
#              imagesetfile,
#              classname,
#              cachedir,
#              ovthresh=0.5,
#              use_07_metric=True):
#     """rec, prec, ap = voc_eval(detpath,
#                            annopath,
#                            imagesetfile,
#                            classname,
#                            [ovthresh],
#                            [use_07_metric])
#     Top level function that does the PASCAL VOC evaluation.
#     detpath: Path to detections
#        detpath.format(classname) should produce the detection results file.
#     annopath: Path to annotations
#        annopath.format(imagename) should be the xml annotations file.
#     imagesetfile: Text file containing the list of images, one image per line.
#     classname: Category name (duh)
#     cachedir: Directory for caching the annotations
#     [ovthresh]: Overlap threshold (default = 0.5)
#     [use_07_metric]: Whether to use VOC07's 11 point AP computation
#        (default True)
#     """
#     # assumes detections are in detpath.format(classname)
#     # assumes annotations are in annopath.format(imagename)
#     # assumes imagesetfile is a text file with each line an image name
#     # cachedir caches the annotations in a pickle file
#     # first load gt
#     if not os.path.isdir(cachedir):
#         os.mkdir(cachedir)
#     cachefile = os.path.join(cachedir, 'annots.pkl')
#     # read list of images
#     # with open(imagesetfile, 'r') as f:
#     #     lines = f.readlines()
#     # imagenames = [x.strip() for x in lines]


#     imagenames = []
#     listdir = os.listdir(osp.join('%s' % args.SIXray_root, 'Annotation'))
#     testdir = listdir[0:6000:5]
#     traindir = set(listdir).difference(set(testdir))
#     traindir = list(traindir)[0:1200]
#     for name in testdir:
#         imagenames.append(osp.splitext(name)[0])


#     if not os.path.isfile(cachefile):
#         # print('not os.path.isfile')
#         # load annots
#         recs = {}
#         for i, imagename in enumerate(imagenames):
#             recs[imagename] = parse_rec(annopath % (imagename),imgpath)
#             '''
#             if i % 100 == 0:
#                 print('Reading annotation for {:d}/{:d}'.format(
#                     i + 1, len(imagenames)))
#             '''
#         # save
#         # print('Saving cached annotations to {:s}'.format(cachefile))
#         with open(cachefile, 'wb') as f:
#             pickle.dump(recs, f)
#     else:
#         # print('no,no,no')
#         # load
#         with open(cachefile, 'rb') as f:
#             recs = pickle.load(f)

#     # print (recs)
#     # print (classname)

#     # extract gt objects for this class
#     class_recs = {}
#     npos = 0
#     for imagename in imagenames:
#         R = [obj for obj in recs[imagename] if obj['name'] == classname]

#         bbox = np.array([x['bbox'] for x in R])
#         difficult = np.array([x['difficult'] for x in R]).astype(np.bool)
#         det = [False] * len(R)
#         npos = npos + sum(~difficult)
#         class_recs[imagename] = {'bbox': bbox,
#                                  'difficult': difficult,
#                                  'det': det}

#     # print (class_recs)

#     # read dets
#     detfile = detpath.format(classname)
#     with open(detfile, 'r') as f:
#         lines = f.readlines()
#     if any(lines) == 1:

#         splitlines = [x.strip().split(' ') for x in lines]
#         image_ids = [x[0] for x in splitlines]
#         confidence = np.array([float(x[1]) for x in splitlines])
#         BB = np.array([[float(z) for z in x[2:]] for x in splitlines])

#         # sort by confidence
#         sorted_ind = np.argsort(-confidence)
#         sorted_scores = np.sort(-confidence)
#         BB = BB[sorted_ind, :]
#         image_ids = [image_ids[x] for x in sorted_ind]

#         # go down dets and mark TPs and FPs
#         nd = len(image_ids)
#         tp = np.zeros(nd)
#         fp = np.zeros(nd)
#         for d in range(nd):
#             R = class_recs[image_ids[d]]
#             bb = BB[d, :].astype(float)
#             ovmax = -np.inf
#             BBGT = R['bbox'].astype(float)
#             if BBGT.size > 0:
#                 # compute overlaps
#                 # intersection
#                 ixmin = np.maximum(BBGT[:, 0], bb[0])
#                 iymin = np.maximum(BBGT[:, 1], bb[1])
#                 ixmax = np.minimum(BBGT[:, 2], bb[2])
#                 iymax = np.minimum(BBGT[:, 3], bb[3])
#                 iw = np.maximum(ixmax - ixmin, 0.)
#                 ih = np.maximum(iymax - iymin, 0.)
#                 inters = iw * ih
#                 uni = ((bb[2] - bb[0]) * (bb[3] - bb[1]) +
#                        (BBGT[:, 2] - BBGT[:, 0]) *
#                        (BBGT[:, 3] - BBGT[:, 1]) - inters)
#                 overlaps = inters / uni
#                 ovmax = np.max(overlaps)
#                 jmax = np.argmax(overlaps)

#             if ovmax > ovthresh:
#                 if not R['difficult'][jmax]:
#                     if not R['det'][jmax]:
#                         tp[d] = 1.
#                         R['det'][jmax] = 1
#                     else:
#                         fp[d] = 1.
#             else:
#                 fp[d] = 1.

#         # compute precision recall
#         fp = np.cumsum(fp)
#         tp = np.cumsum(tp)
#         rec = tp / float(npos)
#         # avoid divide by zero in case the first detection matches a difficult
#         # ground truth
#         prec = tp / np.maximum(tp + fp, np.finfo(np.float64).eps)
#         ap = voc_ap(rec, prec, use_07_metric)
#     else:
#         rec = -1.
#         prec = -1.
#         ap = -1.

#     return rec, prec, ap


# def test_net(save_folder, net, cuda, dataset, transform, top_k,
#              im_size=300, thresh=0.05):
#     # //
#     # //
#     num_images = len(dataset)
#     # all detections are collected into:
#     #    all_boxes[cls][image] = N x 5 array of detections in
#     #    (x1, y1, x2, y2, score)
#     all_boxes = [[[] for _ in range(num_images)]
#                  for _ in range(len(labelmap) + 1)]

#     # timers
#     _t = {'im_detect': Timer(), 'misc': Timer()}
#     output_dir = get_output_dir('ssd300_120000', set_type)
#     det_file = os.path.join(output_dir, 'detections.pkl')

#     for i in range(num_images):
#         im, gt, h, w, og_im = dataset.pull_item(i)
#         # 这里im的颜色偏暗，因为BaseTransform减去了一个mean
#         # im_saver = cv2.resize(im[(a2,a1,0),:,:].permute((a1,a2,0)).numpy(), (w,h))

#         im_det = og_im.copy()
#         im_gt = og_im.copy()

#         # print(im_det)
#         x = Variable(im.unsqueeze(0))
#         if args.cuda:
#             x = x.cuda()
#         _t['im_detect'].tic()
#         detections = net(x).data
#         detect_time = _t['im_detect'].toc(average=False)

#         # skip j = 0, because it's the background class
#         # //
#         # //
#         # print(detections)
#         for j in range(1, detections.size(1)):
#             dets = detections[0, j, :]
#             mask = dets[:, 0].gt(0.).expand(5, dets.size(0)).t()
#             dets = torch.masked_select(dets, mask).view(-1, 5)
#             if dets.size(0) == 0:
#                 continue
#             boxes = dets[:, 1:]
#             boxes[:, 0] *= w
#             boxes[:, 2] *= w
#             boxes[:, 1] *= h
#             boxes[:, 3] *= h
#             # print(boxes)
#             scores = dets[:, 0].cpu().numpy()
#             cls_dets = np.hstack((boxes.cpu().numpy(),
#                                   scores[:, np.newaxis])).astype(np.float32,
#                                                                  copy=False)
#             all_boxes[j][i] = cls_dets

#             # print(all_boxes)
#             for item in cls_dets:
#                 # print(item)
#                 # print(item[5])
#                 if item[4] > thresh:
#                     # print(item)
#                     chinese = labelmap[j - 1] + str(round(item[4], 2))
#                     print(chinese+'det\n\n')
#                     if chinese[0] == '带':
#                         chinese = 'P_Battery_Core' + chinese[6:]
#                     else:
#                         chinese = 'P_Battery_No_Core' + chinese[7:]
#                     cv2.rectangle(im_det, (item[0], item[1]), (item[2], item[3]), (0, 0, 255), 2)
#                     cv2.putText(im_det, chinese, (int(item[0]), int(item[1]) - 5), 0,
#                                 0.6, (0, 0, 255), 2)
#         real = 0
#         if gt[0][4] == 3:
#             real = 0
#         else:
#             real = 1

#         for item in gt:
#             if real == 0:
#                 print('this pic dont have the obj:', dataset.ids[i])
#                 break
#             chinese = labelmap[int(item[4])]
#             # print(chinese+'gt\n\n')
#             if chinese[0] == '带':
#                 chinese = 'P_Battery_Core'
#             else:
#                 chinese = 'P_Battery_No_Core'
#             cv2.rectangle(im_det, (int(item[0] * w), int(item[1] * h)), (int(item[2] * w), int(item[3] * h)),
#                           (0, 255, 255), 2)
#             cv2.putText(im_det, chinese, (int(item[0] * w), int(item[1] * h) - 5), 0, 0.6, (0, 255, 255), 2)
#             # print(labelmap[int(item[4])])

#         print('im_detect: {:d}/{:d} {:.3f}s'.format(i + 1, num_images, detect_time))

#         # cv2.imwrite(f'{SIXray_ROOT}/{dataset.ids[i]}_det.jpg', im_det)
#         # cv2.imwrite(f'{SIXray_ROOT}/{dataset.ids[i]}_gt.jpg', im_gt)
#         # cv2.imwrite('/media/dsg3/husheng/eval/{0}_det.jpg'.format(dataset.ids[i]), im_det)
#         # cv2.imwrite('/media/dsg3/husheng/eval/{0}_gt.jpg'.format(dataset.ids[i]), im_gt)
#     #     break
#     # return
#     #
#     with open(det_file, 'wb') as f:
#         pickle.dump(all_boxes, f, pickle.HIGHEST_PROTOCOL)
    
#     # print('Evaluating detections')
#     evaluate_detections(all_boxes, output_dir, dataset)


# def evaluate_detections(box_list, output_dir, dataset):
#     write_voc_results_file(box_list, dataset)
#     do_python_eval(output_dir)


# def reset_args(EPOCH):
#     global args
#     root = os.getcwd()
#     args.trained_model = "/content/gdrive/My Drive/weights/sixray_9_1_1000.pth"
#     saver_root = root
#     if not os.path.exists(saver_root):
#         os.mkdir(saver_root)
#     args.save_folder = saver_root + '/test/{:d}epoeich_500/'.format(EPOCH)

#     if not os.path.exists(args.save_folder):
#         os.mkdir(args.save_folder)
#     else:
#         shutil.rmtree(args.save_folder)
#         os.mkdir(args.save_folder)

#     global devkit_path
#     devkit_path = args.save_folder


# def xavier(param):
#     nn.init.xavier_uniform_(param)
# def weights_init(m):
#     if isinstance(m, nn.Conv2d):
#         xavier(m.weight.data)
#         m.bias.data.zero_()


# if __name__ == '__main__':
#     # EPOCHS = [5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80]
#     # EPOCHS = [85, 90, 95, 100, 105, 110, 115, 120]
#     # EPOCHS = [90, 95, 100, 105, 110, 115, 120, 125]
#     EPOCHS = [x for x in range(145, 205, 5)]
#     print(EPOCHS)
#     for EPOCH in EPOCHS:
#         reset_args(EPOCH)


#         # load net
#         num_classes = len(labelmap) + 1  # +a1 for background
#         cfg = voc
#         cfg['num_classes'] = 3
#         net = build_ssd('test', 300, num_classes)  # initialize SSD
#         # from ssd_net_vgg import SSD
#         # net = SSD()
#         # net.apply(weights_init)
#         # print(dir(net))
#         # print(net.state_dict)
#         print(args.trained_model)
#         # print(torch.load(args.trained_model))
#         net.load_state_dict(torch.load(args.trained_model))
#         net.eval()

#         # print('Finished loading model!')
#         # load data
#         dataset = SIXrayDetection(args.SIXray_root, args.imagesetfile,
#                                   BaseTransform(300, dataset_mean),
#                                   SIXrayAnnotationTransform(), phase='test')
#         print(args)
#         # args.cuda = False
#         if args.cuda:
#             net = net.cuda()
#             cudnn.benchmark = True
#         # evaluation

#         # with open("./ssd300_120000/test/detections.pkl", 'rb') as fo:  # 读取pkl文件数据
#         #     all_boxes = pickle.load(fo, encoding='bytes')
#         # output_dir = get_output_dir('ssd300_120000', set_type)
#         # evaluate_detections(all_boxes, output_dir, dataset)

#         test_net(args.save_folder, net, args.cuda, dataset,
#                  BaseTransform(net.size, dataset_mean), args.top_k, 300,
#                  thresh=args.confidence_threshold)
#         # if EPOCH == 150:
#         break


In [0]:
import torch
print(torch.cuda.is_available())